In [141]:
# Anal Env.
import pandas as pd
import numpy  as np
import os
import re
import datetime
import pytz
/import copy
from collections import Counter
from emoji       import UNICODE_EMOJI
from functools   import reduce
import sys
sys.path.append('/home/handeully/')
import bigquery_etl as bq
import operator
import time
from IPython.display import display
# NLP Env.
import nltk
from nltk import FreqDist
from nltk.corpus   import stopwords
from nltk.tokenize import regexp_tokenize
from nltk.stem     import WordNetLemmatizer,PorterStemmer,LancasterStemmer
from nltk.corpus   import wordnet
from nltk.corpus   import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

# stop_words = stopwords.words('english')
import spacy
import gensim
from gensim import corpora
from gensim.models.keyedvectors import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
import ray
import psutil
from tqdm.notebook import tqdm

#glove
from glove import Corpus, Glove

# Vis Env.
import pyLDAvis
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# GCP Env.
import google.auth
from google.cloud import bigquery
from googletrans import Translator
from google_trans_new import google_translator

from sklearn.decomposition import TruncatedSVD
from matplotlib import pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import joblib

from sklearn.preprocessing    import Normalizer

# Coding Env.
import warnings
credentials, project_id = google.auth.default(
    scopes=["https://www.googleapis.com/auth/cloud-platform"]
)

client = bigquery.Client(credentials=credentials, project=project_id )
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)
warnings.filterwarnings("ignore", category=DeprecationWarning)

#initialization ray
num_logit_cpus = psutil.cpu_count()
print(f'multiprocessing using {num_logit_cpus} cores')
ray.init(ignore_reinit_error=True,num_cpus=num_logit_cpus)
# ray.shutdown()

2021-06-11 08:32:28,075	INFO worker.py:664 -- Calling ray.init() again after it has already been called.


multiprocessing using 30 cores


In [79]:
# Load Data "FROM" Big Query(db connection)
def convert_lowercase(df):
    df_1 =  df.apply(lambda x: x.astype(str).str.lower() if(x.dtype == 'object') else x)
    return df_1

#check multiprocessing progress 
def to_iterator(obj_ids):
    while obj_ids:
        done, obj_ids = ray.wait(obj_ids)
        yield ray.get(done[0])

def ray_multiprocessing_progress(ray_df):
    for x in tqdm(to_iterator(ray_df), total=len(ray_df)):
        pass
    ray_df  = pd.concat(ray.get(ray_df))
    return ray_df

sql = '''
        select *
        from taxonomy.us_keyword_result
      '''

review_sql = '''
        select *
        from `market-analysis-project-91130`.taxonomy.shng_rvw
      '''

filter_sql =      f''' 
                    SELECT distinct stopword 
                    FROM market-analysis-project-91130.taxonomy.stp_wds;
                   '''

cat_mat_neg_df = convert_lowercase(bq.select_query(sql))
cat_mat_neg_df['count'] = cat_mat_neg_df['count'].astype(int)
print(cat_mat_neg_df.shape)

shiny_df = convert_lowercase(bq.select_query(review_sql))
shiny_df =  shiny_df.drop_duplicates(['title','complimenting_factors'],keep='first')
stopword_df    = convert_lowercase(bq.select_query(filter_sql))
stop_words     = stopword_df['stopword'].tolist()

In [82]:
shiny_df

,date,by,rv,category,sku,product_description,rating,title,reviews,complimenting_factors,ref,complimenting_factor,new_ctgr
0,201905,sophie,az190512137,others,ps-c2824c,"3-step easy pet stairs, grey, x- large",5,excellent product,first of all i have 3 60lb dogs whom are welco...,"design, quality",none,design,others
2,201905,jennie,az190527102,others,ps-c2824c,"3-step easy pet stairs, grey, x- large",5,worth the price to save your back!,these stairs are the reason my dog can sleep w...,"design, quality",jennie) shining review? - marshmallow mountain...,design,others
4,201904,jennie,az190420159,others,ps-d2115s,"pet stairs small, sand",5,excellent quality and price,my chihuahua really loves these steps. he has ...,"value, quality",none,value,others
6,201903,selena,az190401215,others,ps-d2118s,"pet stairs medium, sand",5,great product and great company!,exactly what i was looking for. good quality a...,"quality, customer service, others (greeting card)",none,quality,others
12,201901,none,az190115186,others,olb-bk-2pk,"sleep master headboard bracket, set of 2",5,great packaging,"came with all the necessary hardware, and a ra...","promised customer, others(ratchet)",none,promised customer,others
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2696,201911,selena,az191103180,spring mattresses/sets,az-hbsm-12q,12in gel-infused memory foam hybrid mattress,5,"for the price, it is terrific!",i was looking for a comfortable mattress when ...,quality,none,quality,mattresses
2697,201910,jade,az191004120,spring mattresses/sets,az-hbsm-14k,14in gel-infused memory foam hybrid mattress,5,off to a good start,"i was surprised when they brought it in, all r...",brand comparision,none,brand comparison,mattresses
2698,201908,selena,az190825176,spring mattresses/sets,sm-sc-ebt-13f,ultima comfort 13in deluxe euro box top spring...,5,"so far, so good--1 week and back pain from pre...",got this to replace my atrocious $1000 serta i...,"brand comparison, quality",none,brand comparison,mattresses
2700,201908,sophie,az190816183,spring mattresses/sets,sm-sc-spp-10k,ultima comfort 10in pillow top spring mattress,5,🙌🏾🙌🏾🙌🏾🙌🏾🙌🏾🙌🏾🙌🏾🙌🏾,you know those beds at the hotels?? how comfor...,quality,none,quality,mattresses


In [76]:
factor_sql = '''
                SELECT distinct T2.category,T1.*
                FROM (SELECT a.cmpl_fc1,a.cmpl_fc1_cd,b.cmpl_fc2,c.synonym,c.lemma
                      FROM market-analysis-project-91130.taxonomy.cpl_fc1 a 
                      FULL OUTER JOIN market-analysis-project-91130.taxonomy.cpl_fc2 b   on lower(a.cmpl_fc1_cd) = lower(b.cmpl_fc1_cd)
                      FULL OUTER JOIN market-analysis-project-91130.taxonomy.rvw_kyw_syn c on lower(b.cmpl_fc2) = lower(c.cmpl_fc2) 
                ) T1
                LEFT JOIN market-analysis-project-91130.taxonomy.pdt_ctg T2 ON   T1.cmpl_fc1_cd = T2.cmpl_fc1_cd 
            '''
cpl_factor    = convert_lowercase(bq.select_query(factor_sql))
cmpl_fc_list  = list(set(cpl_factor['cmpl_fc1'].unique().tolist())|set(cpl_factor['cmpl_fc2'].unique().tolist())|set(cpl_factor['synonym'].unique().tolist())|set(cpl_factor['lemma'].unique().tolist()))
multi_express = list(filter(lambda x: len(x.split(' '))>1 , cmpl_fc_list))
# Put this list for MWE Tokenizing
mwe = [tuple(f.split(' ')) for f in multi_express]
print('complain factor    : ' ,cpl_factor.shape)
print(f'all complain factor list  : {len(cmpl_fc_list)}')
print(f'multi_express lenth : {len(mwe)}')

complain factor    :  (4599, 6)
all complain factor list  : 1058
multi_express lenth : 408


In [80]:
@ray.remote
def reviewId_tokenized_sents(df,cat):
    N_POS_TAG   = ['CC','CD','DT','EX','FW','LS','PDT','POS','PRP','PRP$','TO','WDT','WP','WRB']
    Y_POS_TAG   = ['JJ','JJR','JJS','MD','IN','NN','NNS','NNP','NNPS','RB','RBR','RBS','RP','UH','VB','VBG','VBD','VBN','VBP','VBZ']

    keyword_df2 = pd.DataFrame()
    mwe_tokenizer          = nltk.tokenize.MWETokenizer(mwe,separator=' ')
    for i,v in enumerate(df['reviews']):
        corpus = []
        reid = str(df.iloc[i].rv)
        for j in pos_tag(regexp_tokenize(v,"[\w']+")) :
            if (j[1] in Y_POS_TAG ) & (len(j[0])>1) & (j[0].isascii()):
                    corpus.append(j[0])
            tokenized_string       = mwe_tokenizer.tokenize(corpus)
            word_df    = pd.DataFrame(tokenized_string,columns=['word'])
            keyword_df = pd.DataFrame.from_dict(Counter(word_df['word']), orient='index').reset_index()
            keyword_df['reviewId']     = reid
            if keyword_df.shape[0] > 0:
                keyword_df.columns = ['word','count','reviewId']    
        keyword_df2    = pd.concat([keyword_df2,keyword_df])
        keyword_df2    = keyword_df2[~keyword_df2['word'].isin(stop_words)].sort_values(by='reviewId',ascending=False)
        keyword_df2['shiny_class']  = cat
    return keyword_df2

print('================ Count Word DataFrame ===============')
count_df = [reviewId_tokenized_sents.remote(shiny_df[shiny_df['complimenting_factors']==i],i) for i in shiny_df['complimenting_factors'].unique()]
count_df = ray_multiprocessing_progress(count_df)
print('=====================================================')

================ Count Word DataFrame ===============


  0%|          | 0/509 [00:00<?, ?it/s]

In [81]:
count_df

,word,count,reviewId,shiny_class
35,spring,1,az190626163,"design, quality"
10,sleep,1,az190626163,"design, quality"
15,worried,1,az190626163,"design, quality"
34,box,1,az190626163,"design, quality"
33,buy,1,az190626163,"design, quality"
...,...,...,...,...
8,seen,1,az190524064,"packaging, quality, assemble"
5,product,2,az190524064,"packaging, quality, assemble"
4,packaged,2,az190524064,"packaging, quality, assemble"
3,manufactured,1,az190524064,"packaging, quality, assemble"


# LSA

In [85]:
def tf_idf_matrix_all(df, word_all,rv_name) : 
    ## dataframe convert to TF Matrix 
    N = len(df[rv_name].unique())
    
    
    tf = pd.pivot_table(df, values = 'count', index = rv_name, columns = 'word')
    tf = tf.fillna(0)
    
    ## calculating IDF Matrix
    idf = df.groupby(['word']).sum()['count']
    idf = np.log(N/(idf + 1))
    
    # calculating TF-IDF Matrix 
    tfidf = tf * idf
    
    word_current = tf.columns.tolist()
    word_new = set(word_all) - set(word_current)
    
    tfidf_all = tfidf.reindex(columns = [*word_current, *list(word_new)], fill_value = float(0) )
    tfidf_all = tfidf_all[sorted(word_all)] ## rearrange the column names    
    
#     print(tf.shape, idf.shape, tfidf.shape, "-->",tfidf_all.shape)
#     display(tfidf_all.head())
    
    return tfidf_all

word_all = count_df.word.unique().tolist()

tfidf = tf_idf_matrix_all(count_df, word_all,'reviewId')
X = np.array(tfidf)
X.shape

(1207, 5436)

In [186]:
from __future__ import print_function
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from sklearn import metrics

N_POS_TAG  = ['CC','CD','DT','EX','FW','IN','LS','PDT','POS','PRP','PRP$','TO','WDT','WP','WRB']
Y_POS_TAG  = ['JJ','JJR','JJS','MD','NN','NNS','NNP','NNPS','RB','RBR','RBS','RP','UH','VB','VBG','VBD','VBN','VBP','VBZ']

def tokenized_sents(df):
    corpus     = []

    for i in  tqdm(df.reviews.tolist()):
        word = []
        for j in pos_tag(regexp_tokenize(i,r"[\w']+")) :
            if (j[1] in Y_POS_TAG ) & (len(j[0])>1) &  (j[0] not in stop_words ) & (j[0].isascii()):
                word.append(j[0])
#         mwe_tokenizer          = nltk.tokenize.MWETokenizer(mwe,separator=' ')
#         tokenized_string       = mwe_tokenizer.tokenize(word)
        if len(word) >= 1:
            corpus.append(' '.join(word))
        else:
            pass
    return corpus

def tokenized_corpus(df):
    corpus = []
    reviewid = []
    # if you wonder that nltk pos tag
    # nltk.help.upenn_tagset()
    for i,v in enumerate(tqdm(df['reviews'])):
        word = []
        reviewid.append(str(df.iloc[i].rv)) 

        for j in pos_tag(regexp_tokenize(v,"[\w']+")) :
            if (j[1] in Y_POS_TAG ) & (len(j[0])>1) &  (j[0] not in stop_words ) & (j[0].isascii()):
                word.append(j[0])
        tokenized_string       =  ' '.join(word)
        corpus.append(tokenized_string)

    corpus_df = pd.DataFrame({'corpus' : corpus,'reviewId' : reviewid})
    return corpus_df


# shiny_corpus = tokenized_corpus(shiny_df)
# vectorizer = CountVectorizer(min_df = 1, stop_words = stop_words)
# dtm = vectorizer.fit_transform(shiny_corpus['corpus'])
# pd.DataFrame(dtm.toarray(),index=shiny_corpus['reviewId'],columns=vectorizer.get_feature_names()).head(10)
# all_words = vectorizer.get_feature_names()
N_comp = 10

svd_model = TruncatedSVD(n_components = N_comp, algorithm = 'randomized', n_iter = 100, random_state = 1234)
dtm_lsa = svd_model.fit_transform(dtm)
dtm_lsa = Normalizer(copy=False).fit_transform(dtm_lsa)
cols_list= [ 'component_%s'%i for i in range(0,N_comp)]


,component_0,component_1,component_2,component_3,component_4,component_5,component_6,component_7,component_8,component_9
reviewId,,,,,,,,,,
az190512137,0.747566,-0.149202,-0.443901,-0.076786,0.056338,-0.115603,0.118897,0.341841,0.079536,0.249168
az190527102,0.766725,-0.180600,-0.533965,-0.054323,0.181423,-0.151381,-0.072290,0.120559,0.124640,-0.017925
az190420159,0.562729,-0.130745,0.193105,-0.669797,0.072706,-0.237091,-0.195126,-0.073668,0.114202,0.249565
az190401215,0.597469,-0.135505,0.435137,-0.555740,-0.033364,-0.071913,-0.112608,-0.070975,-0.153658,0.280828
az190115186,0.791193,-0.442334,-0.192030,0.084906,-0.302297,0.071539,-0.140587,-0.056117,-0.081334,-0.090774
...,...,...,...,...,...,...,...,...,...,...
az191103180,0.664843,0.537063,-0.126752,-0.035290,-0.280964,0.372013,0.127194,0.059206,0.077209,-0.096213
az191004120,0.338254,0.460560,-0.105628,-0.230002,0.243488,0.717836,0.128934,-0.034778,0.122760,-0.043965
az190825176,0.487781,0.834891,-0.083089,-0.152357,-0.065573,-0.052922,-0.154828,-0.040836,-0.015781,0.043837


In [257]:
# lsa_result = pd.DataFrame(dtm_lsa, index = shiny_corpus['reviewId'], columns = cols_list).reset_index()
# lsa_result = lsa_result.drop_duplicates('reviewId',keep='first')
# lsa_result2 = pd.merge(lsa_result,shiny_corpus.drop_duplicates('reviewId',keep='first'),how='left',on=['reviewId'])
# lsa_result2['class'] = lsa_result2[cols_list].idxmax(axis = 1)
# test = shiny_df.drop_duplicates('rv',keep='first')
# lsa_result2 = pd.merge(lsa_result2,test[['rv','category','reviews','complimenting_factor','complimenting_factors']],how='left',left_on=['reviewId'],right_on=['rv'])

# lsa_result2.groupby('class')['reviewId'].count()
display(lsa_result2[lsa_result2['class']=='component_4'])
for i in lsa_result2[lsa_result2['class']=='component_4'].reviews:
    print(f'{i}\n')


,reviewId,component_0,component_1,component_2,component_3,component_4,component_5,component_6,component_7,component_8,component_9,corpus,class,rv,category,reviews,complimenting_factor,complimenting_factors
328,az190905065,0.474698,0.026782,0.512526,0.464183,0.517444,-0.009706,-0.118402,0.115535,0.013330,-0.020209,open box think going impossible looks pieces b...,component_4,az190905065,box springs,"at first when you open the box you think ""omg ...",assemble,assemble
461,az191209064,0.409516,0.047432,0.531221,0.464293,0.540739,-0.089713,0.080673,-0.063333,0.066082,-0.130195,apple engineers small set tools box,component_4,az191209064,platform beds,its like apple engineers were here! super sma...,quality,"quality, others(ratchet)"


at first when you open the box you think "omg this is going to be impossible!" there is what looks like so many pieces in the box. once you start putting it together though you quickly put 1 and 1, 2 and 2 and next thing you know you're done. everything is labeled properly and you might think no this one is for here but no, simply connect the two numbers that correspond and make sure the top sticker is facing up and within 15/30 minutes you should be done.

 its like apple engineers were here! super small set of "tools" out of the box



In [240]:
lsa_result2.shape

(1207, 13)

In [218]:
similarity = np.asarray(np.asmatrix(dtm_lsa) * np.asmatrix(dtm_lsa).T)
pd.DataFrame(similarity,index=shiny_corpus['reviewId'], columns=shiny_corpus['reviewId'])

reviewId     az190512137  az190527102  az190420159  az190401215  az190115186  \
reviewId                                                                       
az190512137     1.000000     0.907107     0.460285     0.342914     0.645901   
az190527102     0.907107     1.000000     0.452412     0.260639     0.713652   
az190420159     0.460285     0.452412     1.000000     0.904552     0.369792   
az190401215     0.342914     0.260639     0.904552     1.000000     0.413667   
az190115186     0.645901     0.713652     0.369792     0.413667     1.000000   
...                  ...          ...          ...          ...          ...   
az191103180     0.434553     0.384358     0.150065     0.214119     0.402599   
az191004120     0.181684     0.183149     0.091748     0.118541     0.020237   
az190825176     0.268387     0.275487     0.301471     0.267881     0.057047   
az190816183     0.639332     0.544158     0.675043     0.535417     0.182182   
az191012135     0.403336     0.450272     0.878965     0.805048     0.325281   

reviewId     az190621226  az190623232  az190630242  az190413172  az190117175  \
reviewId                                                                       
az190512137     0.201072     0.381712     0.048917     0.126081     0.188939   
az190527102     0.153316     0.253632     0.152274     0.277521     0.090528   
az190420159     0.626890     0.713569     0.699385     0.826624     0.413627   
az190401215     0.749157     0.748462     0.713459     0.788368     0.405379   
az190115186     0.409800     0.278515     0.306393     0.367454     0.069712   
...                  ...          ...          ...          ...          ...   
az191103180     0.214329     0.218237     0.025013     0.155283     0.196392   
az191004120     0.063384     0.120408    -0.030364     0.076692     0.102206   
az190825176     0.080151     0.107342     0.030366     0.264224     0.087723   
az190816183     0.347852     0.565373     0.299142     0.426873     0.472763   
az191012135     0.674991     0.752218     0.774188     0.889838     0.528102   

reviewId     az190523159  az190525136  az191012163  az191111180  az190303187  \
reviewId                                                                       
az190512137     0.138371     0.391684     0.083326     0.363550     0.532912   
az190527102     0.219896     0.407556     0.121006     0.323050     0.684830   
az190420159     0.716153     0.803645     0.114676     0.392802     0.691249   
az190401215     0.813949     0.680915     0.200074     0.382730     0.509700   
az190115186     0.345249     0.324964     0.094829     0.167064     0.588203   
...                  ...          ...          ...          ...          ...   
az191103180     0.201078     0.309752     0.458761     0.657526     0.227087   
az191004120     0.141725     0.182186     0.869761     0.865854     0.092881   
az190825176     0.258459     0.369254     0.126800     0.392546     0.195501   
az190816183     0.377195     0.574585     0.109151     0.487822     0.395426   
az191012135     0.837393     0.830316     0.339736     0.621827     0.698757   

reviewId     az190126187  az190819234  az190627193  az190203180  az190914156  \
reviewId                                                                       
az190512137     0.022721     0.643057     0.785157     0.922991     0.219902   
az190527102     0.126361     0.775909     0.731998     0.905215     0.275367   
az190420159     0.443078     0.725878     0.495321     0.529005     0.557458   
az190401215     0.557941     0.508236     0.452900     0.362583     0.504253   
az190115186     0.248985     0.619455     0.711071     0.584649     0.291322   
...                  ...          ...          ...          ...          ...   
az191103180     0.138704     0.226156     0.640814     0.354469     0.381999   
az191004120     0.061272     0.053242     0.211640     0.164782     0.601589   
az190825176     0.152715     0.205748     0.381136     0.271733     0.198381 

In [220]:
review_list = ['az190420159','az190413172','az190117175']

text_dd = shiny_df[shiny_df['rv'].isin(review_list)].reset_index(drop=True)
for i in range(0,len(text_dd)):
    print(f'■ reviewId : {text_dd.rv.iloc[i]} \n■ classs :{text_dd.complimenting_factor.iloc[i]}\n■ contents :{text_dd.reviews.iloc[i]} \n')

■ reviewId : az190420159 
■ classs :value
■ contents :my chihuahua really loves these steps. he has a neck problem and is not supposed to jump up and down from furniture. the price is great. we shopped at pet smart and couldn’t find anything comparable for under $50. very well made. 

■ reviewId : az190413172 
■ classs :design
■ contents :great product!  the form is firm yet flexible and will more than likely obtain its shape.  my mini schnauzers would not use the harder plastic steps so i tried this one and they both, although a little hesitant at first, love it now.  i even caught one of them taking a nap on the bottom step yesterday  the fabric design is adorable as well !! 

■ reviewId : az190117175 
■ classs :quality
■ contents :i like that this step/stair can be used as a two step or three step as well as the fact that the first step horizontally is larger at 11.9" vs 7" on zinus's other stair/step/ramp models. the foam is firm and solid enough in my dogs experience, that it can 

## Truncated SVD 

In [93]:


n_comp = [4,10,20,30,40,50,100,150,200,300,400,500,600,700,800,900,1000,1500,2000,2500]

var_explained  = []
for i in n_comp : 
    svd_model = TruncatedSVD(n_components = i, algorithm = 'randomized', n_iter = 100, random_state = 1234)
    svd_model.fit(X)
    var_explained.append(svd_model.explained_variance_ratio_.sum())
    print(f"Number of components = {i} and explained variance = {svd_model.explained_variance_ratio_.sum()}")
    
plt.plot(n_comp[:len(var_explained)], var_explained)
plt.xlabel('Number of components')
plt.ylabel('Explained Variance')
plt.title('Plot : The relation between number of components and explained variance ')
plt.show() 

# Number of components = 4 and explained variance = 0.03069412707066218
# Number of components = 10 and explained variance = 0.05078730503906526
# Number of components = 20 and explained variance = 0.07596743718433414
# Number of components = 30 and explained variance = 0.09747704894357086
# Number of components = 40 and explained variance = 0.11679129055991289
# Number of components = 50 and explained variance = 0.13470440315528953
# Number of components = 100 and explained variance = 0.21136225427676691
# Number of components = 150 and explained variance = 0.27422044315281624
# Number of components = 200 and explained variance = 0.3281364560757497
# Number of components = 300 and explained variance = 0.4173923232897509
# Number of components = 400 and explained variance = 0.4890907536255331
# Number of components = 500 and explained variance = 0.5484204792009648
# Number of components = 600 and explained variance = 0.5985218954267318
# Number of components = 700 and explained variance = 0.6413089969186242
# Number of components = 800 and explained variance = 0.6783836427262188
# Number of components = 900 and explained variance = 0.7107083156897218
# Number of components = 1000 and explained variance = 0.7391278616334593
# Number of components = 1500 and explained variance = 0.8399643334538267
# Number of components = 2000 and explained variance = 0.8994683351325431
# Number of components = 2500 and explained variance = 0.9364499027101708

Number of components = 4 and explained variance = 0.0389894823744438
Number of components = 10 and explained variance = 0.07692867456169904
Number of components = 20 and explained variance = 0.12053510546932197
Number of components = 30 and explained variance = 0.15578629499027458
Number of components = 40 and explained variance = 0.18752286540215798
Number of components = 50 and explained variance = 0.21676335436753466
Number of components = 100 and explained variance = 0.3391968988369747
Number of components = 150 and explained variance = 0.43529046518689646
Number of components = 200 and explained variance = 0.5148832629548454
Number of components = 300 and explained variance = 0.6405088622397459


KeyboardInterrupt: 

In [111]:
def save_model(mdl,mdl_file_nm):
    global model_dir,model_name
    DIR = MODEL_DIR
    model_name = mdl_file_nm
    model = joblib.dump(mdl,open(DIR+model_name+".pkl",'wb'))
    model_dir = DIR+model_name
    print('%s 경로에 %s 모델 저장완료 '%(DIR,model_name+".pkl"))
    return model_name
    
#save model
def load_model(filename):
    model_dir  = MODEL_DIR+filename
    load_model = joblib.load(model_dir+".pkl")
    return load_model

# MODEL_DIR  = '/home/yeomsara/YSR/model/'
# mdl_name = 'us_mat_keyword_LSA_Model'
# save_model(svd_model,mdl_name)

509

# TruncatedSVD

In [145]:
svd_model = TruncatedSVD(n_components = 5, algorithm = 'randomized', n_iter = 30, random_state = 1234)
svd_model.fit(X)

TruncatedSVD(n_components=5, n_iter=30, random_state=1234)

In [154]:
X.shape

(1207, 5436)

In [117]:
# svd_model = load_model(mdl_name)

## 각 행렬의 Matrix shape 확인
U = svd_model.transform(X)
S = svd_model.singular_values_
VT = svd_model.components_
print(U.shape, S.shape, VT.shape)

## 각 토픽의 분산 설명력
var_exp = svd_model.explained_variance_ratio_
print("explained variance ratio : ", svd_model.explained_variance_ratio_.sum())
# print(var_exp)

(1207, 509) (509,) (509, 5436)
explained variance ratio :  0.8127381028722367


In [157]:
# dtm_lsa = svd_model.fit_transform(X)
# dtm_lsa = Normalizer(copy=False).fit_transform(dtm_lsa)

pd.DataFrame(svd_model.components_,index =cols_list,columns = word_all).T

,component_0,component_1,component_2,component_3,component_4
spring,0.000530,-0.001504,-0.002187,-0.001978,0.001667
sleep,0.002054,-0.000640,-0.001155,-0.000481,-0.000599
worried,0.000715,-0.000518,-0.000933,-0.000989,0.000336
box,0.001070,-0.000478,0.001700,0.001767,-0.002950
buy,0.002418,-0.003059,0.004407,0.001264,0.001836
...,...,...,...,...,...
sanded,0.006904,-0.001579,0.000029,0.004478,-0.007957
hella,0.004287,0.000731,0.000062,0.001496,-0.004843
max,0.002075,-0.000316,-0.000356,0.001289,-0.000382
forbid,0.000502,-0.000785,-0.001922,-0.001691,0.000725


In [155]:
svd_model.components_.shape

(5, 5436)

In [90]:
# Compute document similarity using LSA components
similarity = np.asarray(np.asmatrix(dtm_lsa) * np.asmatrix(dtm_lsa).T)
lsa_similarity_df = pd.DataFrame(similarity,index=tfidf.index.tolist(), columns=tfidf.index.tolist())

In [91]:
# lsa_similarity_df.T
lsa_similarity_df

az190101055   az190101097   az190101142   az190101174  \
az190101055     1.000000  4.662088e-02  1.451694e-02  2.541992e-02   
az190101097     0.046621  1.000000e+00 -2.368982e-17  1.252269e-02   
az190101142     0.014517 -2.368982e-17  1.000000e+00  2.246782e-02   
az190101174     0.025420  1.252269e-02  2.246782e-02  1.000000e+00   
az190102047     0.032039  2.158306e-02 -1.512462e-17  5.377413e-03   
...                  ...           ...           ...           ...   
tmp4            0.014057  8.070584e-03  2.252441e-02  5.932265e-02   
tmp5            0.035299  1.243226e-02 -3.038477e-17 -3.426079e-17   
tmp6            0.039112  9.795993e-02 -8.619407e-18  1.350745e-02   
zn190518004     0.052356  2.938863e-02  4.006737e-02  1.174619e-02   
zn190706001     0.000377  2.616129e-02  2.472986e-02  4.900304e-03   

              az190102047   az190102048   az190102051   az190102066  \
az190101055  3.203855e-02  1.645094e-02  9.771310e-03 -1.091792e-16   
az190101097  2.158306e-02  1.789846e-02  7.650602e-03  3.200204e-02   
az190101142 -1.512462e-17  2.775558e-17 -9.540979e-17 -5.721877e-17   
az190101174  5.377413e-03  5.811324e-17 -1.734723e-18  2.486913e-02   
az190102047  1.000000e+00  2.808313e-03  7.979728e-17  7.786930e-03   
...                   ...           ...           ...           ...   
tmp4         6.868422e-03 -1.084202e-18  2.791078e-02  7.700510e-03   
tmp5         2.808590e-03  5.934678e-03  8.239937e-18  7.622882e-03   
tmp6         3.853136e-02  2.775558e-17  1.027824e-16  2.998748e-02   
zn190518004  1.565631e-02  5.156465e-02  2.356927e-02  8.193015e-03   
zn190706001  1.251890e-03  7.113836e-02  2.714762e-02  7.096037e-03   

              az190102116   az190102127   az190102146   az190102227  \
az190101055  1.557366e-02  4.463170e-02  6.867604e-02  1.534926e-02   
az190101097  3.424287e-02  3.963612e-03 -6.808790e-17  1.086165e-02   
az190101142 -2.352990e-16 -1.968911e-16  8.144890e-03  2.790736e-16   
az190101174 -1.090707e-16  1.084202e-17  4.570864e-02 -4.033232e-17   
az190102047  3.037166e-02  5.644922e-03  3.530077e-03  3.162842e-02   
...                   ...           ...           ...           ...   
tmp4         1.457710e-16  9.567761e-05  2.438999e-02  1.533639e-02   
tmp5         8.150492e-03  2.240592e-03  8.562925e-05  2.818706e-02   
tmp6         5.642699e-02  1.506861e-04  2.083740e-04  2.415383e-02   
zn190518004  3.041260e-02  9.606078e-03  8.238378e-02  2.195962e-02   
zn190706001  5.085982e-04  5.290907e-17  9.907283e-03  1.323138e-02   

              az190103104   az190103111   az190104058   az190104072  \
az190101055  1.360192e-02  2.824799e-02  4.780219e-02  1.333021e-01   
az190101097  6.121725e-03 -8.413409e-17  8.008294e-02  1.810081e-02   
az190101142 -9.204876e-17  1.691355e-16  1.170938e-17  3.342731e-17   
az190101174  8.673617e-18  3.102741e-02  7.849624e-17  1.855822e-02   
az190102047  2.553770e-02  2.644902e-02  9.813437e-02  5.867586e-02   
...                   ...           ...           ...           ...   
tmp4         1.747904e-04 -8.521829e-17 -3.686287e-18  9.778820e-03   
tmp5         4.549030e-02  4.416100e-02  1.704949e-02  1.574916e-02   
tmp6         9.322701e-03 -6.071532e-17  4.352710e-02  2.395291e-02   
zn190518004  7.513521e-17  3.864873e-03 -4.517057e-17  5.569060e-03   
zn190706001  3.498813e-03  1.099286e-02 -6.353425e-17  1.547970e-16   

              az190104179   az190104184  az190105044  az190105093  \
az190101055  2.374889e-02  1.714992e-02     0.002795     0.051247   
az190101097  2.134377e-02  2.418622e-02     0.006454     0.076648   
az190101142 -5.876376e-17 -4.125389e-17     0.027984     0.008173   
az190101174  6.303954e-03  5.300580e-03     0.025136     0.005515   
az190102047  2.112965e-03  4.739224e-02     0.030062     0.048789   
...                   ...           ...          ...          ...   
tmp4        -1.113205e-16 -1.577650e-16     0.000094     0.009428   
tmp5         3.100847e-03  1.750283e-02     0.016807     0.04

In [60]:
# reid = 'R1UPUIWZH23YZ1'.lower()
# aa   = lsa_similarity_df[reid].T.reset_index()
# aa.columns    = ['reviewId','similarity']
# aa['reviewId']   = aa['reviewId'].apply(lambda x : x.upper())
aa[aa['similarity'] > 0.3].reviewId.tolist()

['R1SFKTARWCBYQO',
 'R1UPUIWZH23YZ1',
 'R2G0YUBXR6J4MN',
 'R3CEGV37C8S4YQ',
 'R9YNLC14MBQMM']